In [31]:
# Jupyter Notebook libraries
from IPython.display import display
from ipywidgets import widgets

# Conda libraries
import pandas as pd
import datetime as dt

# External libraries
import mplfinance as mpf


# Local libraries
from utils.alpaca import alpaca as alp

## Load list of active assets

In [32]:
active_assets = alp.list_assets(status='active')
active_assets_df = pd.DataFrame([asset._raw for asset in active_assets if asset._raw['tradable'] == True])
active_assets_df = active_assets_df.loc[lambda df: (df['class'] == 'us_equity')]
active_assets_df.drop(columns=['class', 'status', 'tradable', 'min_order_size', 'min_trade_increment', 'price_increment'], inplace=True)
# active_assets_df.set_index(keys='id', drop=True, inplace=True)
display(active_assets_df.columns)

Index(['id', 'exchange', 'symbol', 'name', 'marginable', 'shortable',
       'easy_to_borrow', 'fractionable'],
      dtype='object')

In [33]:
# asset_class_menu = widgets.Dropdown(options=active_assets_df['class'].unique())
exchange_menu = widgets.Dropdown(options=active_assets_df['exchange'].unique())
asset_menu = widgets.Dropdown(options=active_assets_df['name'].sort_values().unique())

asset_detail = widgets.Label(asset_menu.value)
asset_price = widgets.Label()
# asset_history = widgets.Textarea(layout={'height':'100px', 'width':'100%'})

In [34]:
today = dt.datetime.now()
offset = max(1, (today.weekday() + 6) % 7 -3)
timedelta = dt.timedelta(offset)
most_recent = today - timedelta
print(f"Most recent trading day: {most_recent.date()}")


Most recent trading day: 2022-05-09


In [36]:
numdays=365
oldest_date = most_recent - dt.timedelta(days=numdays)

def get_last_price(symbol):
    return alp.get_bars(
        symbol=symbol,
        timeframe='1D', 
        # limit=100, 
        end=str(most_recent.date()),
        start=str(oldest_date.date()),
    ).df

In [37]:
def display_asset_menu(
    # asset_class, 
    exchange, 
    asset,
):
    filtered_asset_list = active_assets_df.loc[lambda df: (df['exchange'] == exchange)]['name'].sort_values().unique()

    asset_menu.options = filtered_asset_list
    selected_asset = active_assets_df.loc[lambda df: (df['name'] == asset_menu.value)]

    asset_detail.value = f"{selected_asset.iloc[0]['symbol']}: {selected_asset.iloc[0]['name']}"

    # Use the Alpaca get_barset function to get current closing prices the portfolio
    prices_most_recent_df = get_last_price(selected_asset.iloc[0]['symbol'])
    # prices_most_recent_df = alp.get_bars(
    #     selected_asset.iloc[0]['symbol'],
    #     # tickers, 
    #     timeframe='1D', 
    #     # limit=100, 
    #     end=str(most_recent.date()),
    #     start=str(oldest_date.date()),
    #     ).df
    # display(prices_most_recent_df.index.date())
    asset_price.value = f"Close Price: ${prices_most_recent_df.iloc[-1]['close']}"
    # asset_history.value = f"{prices_most_recent_df}"
    plot = mpf.plot(prices_most_recent_df, type='candle', volume=True)
    

In [38]:
asset_plot = widgets.interactive_output(
    display_asset_menu,
    {
        # 'asset_class':asset_class_menu, 
        'exchange': exchange_menu, 
        'asset': asset_menu,
    }
)

In [39]:
asset_df = pd.DataFrame()
amount = widgets.FloatText(description="Amount")


In [40]:
def add_asset():
    # temp_df = pd.DataFrame(active_assets_df.loc[lambda df: (df['name'] == asset_menu.value)])
    selected_asset = active_assets_df.loc[lambda df: (df['name'] == asset_menu.value)]
    selected_asset['amount'] = amount.value

    prices_most_recent_df = get_last_price(selected_asset.iloc[0]['symbol'])

    selected_asset['last_close'] = prices_most_recent_df.iloc[-1]['close']

    selected_asset['current_value'] = selected_asset['last_close'] * selected_asset['amount']

    global asset_df
    asset_df = asset_df.append(selected_asset, ignore_index=True)


In [41]:
button = widgets.Button(description="Add Asset")
portfolio_summary = widgets.Output()
def on_button_clicked(b):
    add_asset()
    amount.value = 0
    with portfolio_summary:
        portfolio_summary.clear_output()
        # display(temp_df)
        display(asset_df.loc[:,['exchange', 'symbol', 'name', 'amount', 'last_close', 'current_value']])
  

button.on_click(on_button_clicked)
# display(button, output)

In [42]:
chooser_tab = widgets.VBox(
    [
        widgets.HBox(
            [
                exchange_menu, 
                asset_menu,
                amount,
                button,
            ]
        ),
        asset_price,
        widgets.HBox(
            [
                asset_plot,
                portfolio_summary,
            ]
        )
    ]
)

In [43]:
tab_list = ['Chooser', 'Portfolio']
children = [chooser_tab, portfolio_summary]
tab = widgets.Tab(children=children)
for i, title in enumerate(tab_list):
    tab.set_title(i, title)

In [44]:
display(tab)

In [45]:
# display(widgets.HBox([exchange_menu, asset_menu, asset_price, button, amount]), asset_plot)